In [ ]:
import pandas as pd
from freesurfer_stats import CorticalParcellationStats
import glob
import re
import itertools
import numpy as np
import pyls
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
import pickle
from sklearn import preprocessing
from scipy.stats import zscore
from nilearn import input_data
from nilearn import plotting
import pingouin as pg

In [ ]:
sns.set_context("notebook", font_scale=1.7)
sns.set_style("whitegrid")

## Carga datos volúmenes

In [ ]:
all_volumes_file = 'data/all_volumes.csv'
all_volumes = pd.read_csv(all_volumes_file)
all_volumes.drop(columns=all_volumes.columns[0], axis=1, inplace=True)
#all_volumes.columns.to_list()

# Selección de estructuras, no se inlcluye el 5to ventrículo
columns = list(range(0,15)) + list(range(21,54)) + list(range(61,467))
selected_values = all_volumes.iloc[:,columns]

# Cambio nombre grupo
selected_values.rename(columns={'grupo': 'group'}, inplace=True)

# Selección prisma 1
selected_values = selected_values.loc[selected_values['project'].isin(['prisma1','controles'])]

# Selección TAB y CTRL
selected_values = selected_values.loc[selected_values['group'].isin(['TAB','CTR'])]
#selected_values

In [ ]:
# Conteo datos iniciales
selected_values.groupby('group').describe()

In [ ]:
# Selección sujetos chequeo freesurfer
check_file = 'data/ConsolidadoRatings.xlsx'
check_data = pd.read_excel(check_file,'ALL')
suj_to_remove = check_data[check_data['Rating'] == 'FaIL'].Codigo

selected_values = selected_values.set_index('subject')
selected_values.drop(index=suj_to_remove,inplace=True)
selected_values = selected_values.reset_index(drop=False)
selected_values.groupby('group').describe()

## Normalización por volumen intracraneal

In [ ]:
selected_values.iloc[:,3:] = selected_values.iloc[:,3:].div(selected_values["Estimated Total Intracranial Volume"], axis=0)
#selected_values

In [ ]:
all_data = selected_values
# Remover columna de ETIV
all_data = all_data.drop(['Estimated Total Intracranial Volume'],axis=1)
#all_data

## Carga de datos clínicos

In [ ]:
clinical_file = 'data/BFCN_t1_cov.xlsx'
clinical_data = pd.read_excel(clinical_file)
clinical_data.loc[clinical_data.subject.str.contains('CTRL'), 'project'] = 'controles'
# Selección de variables
clinical_data.drop(columns=['group','columna1','dim3','dim4','id_prisma'],inplace=True)
#clinical_data

## Merge both files

In [ ]:
# Unión archivos volúmenes e información clínica
clinical_image_df= pd.merge(all_data, clinical_data, how='left', on=['subject', 'project'])
clinical_image_df.groupby('group').describe()

In [ ]:
#Remove data for TAB and diagnosis 2
clinical_image_df.drop(clinical_image_df.index[clinical_image_df['diagnosis']=='Bipolar Affective Disorder 2'],inplace=True)
clinical_image_df.groupby('group').describe()

In [ ]:
clinical_image_df.head()

## Normalización de los datos

In [ ]:
# Normalización z-score por fila (sujeto) para datos de volumen 3 a 454
clinical_image_z = clinical_image_df
clinical_image_z.iloc[:,3:453] = clinical_image_z.iloc[:,3:453].apply(zscore, axis=1) 
clinical_image_z.groupby('group').describe()

In [ ]:
#clinical_image_z.iloc[:,453:].columns.to_list()

In [ ]:
# Selección variables clínicas
list_regression = ['age','sex','education_level','etiv',
                   'antipsychotic_use','antidepressant_use','moodstabiliser_use']
list_norm = ['age','sex','education_level','etiv']

In [ ]:
# Eliminación datos nan
clinical_image_z_na = clinical_image_z.dropna(subset = list_regression)
clinical_image_z_na.reset_index(drop= True, inplace=True)
clinical_image_z_na.groupby('group').describe()

In [ ]:
# Normalización covariables por columna
clinical_image_z_na[list_norm] = clinical_image_z_na[list_norm].apply(zscore, axis=0)

## Regresión lineal múltiple

In [ ]:
vol_cols = clinical_image_z_na.iloc[:,3:453].columns.to_list()
residuals = pd.DataFrame()

for col in vol_cols:
    lm = pg.linear_regression(clinical_image_z_na[list_regression]
                              ,clinical_image_z_na[col])
    
    residuals[col] = lm.residuals_


In [ ]:
# Formación nuevo data frame
clinical_image_z_na.iloc[:,3:453] = residuals.values

## PLS BEHAVIORAL

In [ ]:
#clinical_image_z.iloc[:,453:].columns.to_list()

In [ ]:
# Selección variables
# Variables psycológicas que se encuentran para TAB y CTRL (continuas)
y_list = ['psy_cit','psy_tmt_a','psy_tmt_b',
          'psy_tavec_1_en_a','psy_tavec_a_5_ens','psy_tavec_rec_tl_a','psy_tavec_b_in','psy_tavec_%_pri',
          'psy_tavec_%_med','psy_tavec_%_rec','psy_tavec_re_lb_cp','psy_tavec_re_lb_lp','psy_tavec_re_cl_cp',
          'psy_tavec_re_cl_lp','psy_tavec_est_ser_a','psy_tavec_est_ser_rcp',
          'psy_tavec_est_ser_rlp','psy_tavec_est_sem_a','psy_tavec_est_sem_b','psy_tavec_rec','psy_wmsiii_d',
          'psy_wmsiii_i','psy_fas_sm','psy_fas_fn','psy_wcst_err','psy_wcst_cat','psy_wcst_%pers','psy_wcst_%conc']

In [ ]:
# Eliminación datos nan
clinical_image_z_na = clinical_image_z.dropna(subset = y_list)
clinical_image_z_na.reset_index(drop= True, inplace=True)
clinical_image_z_na.groupby('group').describe()

In [ ]:
# Normalización covariables por columna
clinical_image_z_na[y_list] = clinical_image_z_na[y_list].apply(zscore, axis=0)

In [ ]:
all_data_sorted = clinical_image_z_na.sort_values('group', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

count_groups = all_data_sorted.groupby(['group']).count()
groups_dict = count_groups['subject'].to_dict()

list_len = list(groups_dict.values())
groups = list(groups_dict.keys())
groups_dict

In [ ]:
# Matriz X: datos de volúmenes
X_brain = all_data_sorted.iloc[:,3:453].values

In [ ]:
Y_beha = all_data_sorted[y_list].values

In [ ]:
pls = pyls.behavioral_pls(X_brain, Y_beha, groups=list_len, seed=42, n_proc='max')

In [ ]:
# Significancia PLS 
pvals = pls.permres['pvals']
x_weights = pls.bootres.x_weights_normed 
y_weights = pls.y_weights
pvals

In [ ]:
sig_ind = np.where(pvals < 0.05)[0]
sig_ind

In [ ]:
pls.varexp[sig_ind]

In [ ]:
conditions = []
for j in y_list:
    conditions = conditions + [j + '-' + i for i in groups]
#conditions

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5,25))
icx = 0
err = (pls["bootres"]["y_loadings_ci"][:, icx, 1] - pls["bootres"]["y_loadings_ci"][:, icx, 0]) / 2   
axs.barh(np.arange(len(err)), pls["y_loadings"][:, icx], xerr=err)
axs.set_yticks(np.arange(len(conditions)))#, labels=ext_pet_roi_df.columns[1:].to_numpy()[relidx])
axs.set_yticklabels(np.array(conditions)) #(ext_pet_roi_maps.columns[1:].to_numpy()[sorted_idx])

In [ ]:
# Gráfico que condiciones que no pasan por cero
icx = 0
ind_sig = np.where(abs(err) < abs(pls["y_loadings"][:, icx]))[0]

new_ind = list()
for i in range(len(ind_sig)):
    if ind_sig[i] % 2 == 0:
        if ind_sig[i] + 1 == ind_sig[i+1]:
            print('ok')
        else:
            new_ind.insert(i+1, ind_sig[i] + 1 )
            print('no')
    else:
        if ind_sig[i] - 1 == ind_sig[i-1]:
            print('ok')
        else:
            new_ind.insert(i-1, ind_sig[i] - 1 )
            #print('no')
            
new_sig = list(ind_sig) + new_ind
new_sig.sort()

# Adicionar ambas condiciones (generar lista)
#ind_sig = [0, 1, 24, 25, 26, 27, 30, 31, 32, 33, 34, 35, 38, 39, 40, 41, 44, 45, 46, 47, 48, 49, 50, 51] 

color_list = ['olive','gray']

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5,10))
icx = 0
err_sig = (pls["bootres"]["y_loadings_ci"][new_sig, icx, 1] - pls["bootres"]["y_loadings_ci"][new_sig, icx, 0]) / 2   
axx= axs.barh(np.arange(len(err_sig)), pls["y_loadings"][new_sig, icx], xerr=err_sig, color=color_list)
conditions_new = [i.split('-')[0] for i in conditions]
axs.set_yticks(np.arange(len(new_sig)))
axs.set_yticklabels(np.array(conditions_new)[new_sig])
axs.legend(axx,['ctrl','tab'])

## Gráficos de dispersión

In [ ]:
score_pls = pd.DataFrame()
score_pls['subject'] = all_data_sorted['subject']
score_pls['group'] = all_data_sorted['group']
score_pls['psy_tavec_re_cl_lp'] = all_data_sorted['psy_tavec_re_cl_lp']
score_pls['psy_tavec_re_cl_cp'] = all_data_sorted['psy_tavec_re_cl_cp']
score_pls['x_score']=pls.x_scores[:,0]
score_pls['y_score']=pls.y_scores[:,0]

In [ ]:
# Gráfico x vs y scores
sns.set(rc={'figure.figsize':(11,8)})
g=sns.scatterplot(x="x_score", y="y_score",
                  hue="group",data=score_pls,linewidth=1)

plt.title("X SCORE VS Y SCORE LV 1")
plt.show()

In [ ]:
# Gráfico variable latente 1 vs psy_tavec_re_cl_cp (recuerdo claves a corto plazo)
sns.set(rc={'figure.figsize':(11,8)})
        
g=sns.scatterplot(x="x_score", y="psy_tavec_re_cl_cp",
                  hue="group",legend=True,data=score_pls,linewidth=1)

plt.title("X score vs tavec_re_cl_cp ")
plt.show()

In [ ]:
# Gráfico variable latente 1 vs psy_tavec_re_cl_cp (recuerdo claves a corto plazo)
sns.set(rc={'figure.figsize':(11,8)})
        
g=sns.scatterplot(x="y_score", y="psy_tavec_re_cl_cp",
                  hue="group",legend=True,data=score_pls,linewidth=1)

plt.title("Y score vs tavec_re_cl_cp ")
plt.show()

In [ ]:
# Gráfico variable latente 1 vs psy_tavec_re_cl_lp (recuerdo claves a largo plazo)
sns.set(rc={'figure.figsize':(11,8)})
        
g=sns.scatterplot(x="x_score", y="psy_tavec_re_cl_lp",
                  hue="group",legend=True,data=score_pls,linewidth=1)

plt.title("X score vs tavec_re_cl_lp ")
plt.show()

In [ ]:
# Gráfico variable latente 1 vs psy_tavec_re_cl_lp (recuerdo claves a largo plazo)
sns.set(rc={'figure.figsize':(11,8)})
        
g=sns.scatterplot(x="y_score", y="psy_tavec_re_cl_lp",
                  hue="group",legend=True,data=score_pls,linewidth=1)

plt.title("Y score vs tavec_re_cl_lp ")
plt.show()

## PLS invirtiendo X: behavior, Y: volumes

In [ ]:
# Matriz X: datos de volúmenes
Y = all_data_sorted.iloc[:,3:453].values
X = all_data_sorted[y_list].values

In [ ]:
pls2 = pyls.behavioral_pls(X, Y, groups=list_len, seed=42, n_proc='max')

In [ ]:
pvals2 = pls2.permres['pvals']
x_weights = pls2.bootres.x_weights_normed # 28 
y_weights = pls2.y_weights # 900 (450 características por grupo)
pvals2

In [ ]:
sig_ind = np.where(pvals2 < 0.05)[0]
sig_ind

In [ ]:
pls.varexp[sig_ind]

In [ ]:
conditions2 = all_data_sorted.iloc[:,3:453].columns.to_list()
#conditions2

In [ ]:
conditions = []
for j in conditions2:
    conditions = conditions + [j + '-' + i for i in groups]
#conditions

In [ ]:
# Gráfico que condiciones que no pasan por cero
icx = 1
err = (pls2["bootres"]["y_loadings_ci"][:, icx, 1] - pls2["bootres"]["y_loadings_ci"][:, icx, 0]) / 2   
ind_sig = np.where(abs(err) < abs(pls2["y_loadings"][:, icx]))[0]

color_list = ['olive','gray']

In [ ]:
new_ind = list()
for i in range(len(ind_sig)):
    if ind_sig[i] % 2 == 0:
        if ind_sig[i] + 1 == ind_sig[i+1]:
            print('ok')
        else:
            new_ind.insert(i+1, ind_sig[i] + 1 )
            print('no')
    else:
        if ind_sig[i] - 1 == ind_sig[i-1]:
            print('ok')
        else:
            new_ind.insert(i-1, ind_sig[i] - 1 )
            #print('no')
            
new_sig = list(ind_sig) + new_ind
new_sig.sort()

In [ ]:
# División estructuras subcorticales
sub_idx = np.where(np.array(new_sig) < 100)[0]
sub_sig = np.array(new_sig)[sub_idx.astype(int)]
newfig, axs = plt.subplots(1, 1, figsize=(5,15))
err_sig = (pls2["bootres"]["y_loadings_ci"][sub_sig, icx, 1] - pls2["bootres"]["y_loadings_ci"][sub_sig, icx, 0]) / 2   
axx= axs.barh(np.arange(len(err_sig)), pls2["y_loadings"][sub_sig, icx], xerr=err_sig, color=color_list)
conditions_new = [i.split('-')[0:-1] for i in conditions]
#conditions_new = conditions
axs.set_yticks(np.arange(len(sub_sig)))
axs.set_yticklabels(np.array(conditions_new)[sub_sig])
axs.legend(axx,['ctrl','tab'])

In [ ]:
cor_idx = np.where(np.array(new_sig) > 100)[0]
cor_sig = np.array(new_sig)[cor_idx.astype(int)]
newfig, axs = plt.subplots(1, 1, figsize=(5,70))
err_sig = (pls2["bootres"]["y_loadings_ci"][cor_sig, icx, 1] - pls2["bootres"]["y_loadings_ci"][cor_sig, icx, 0]) / 2   
axx= axs.barh(np.arange(len(err_sig)), pls2["y_loadings"][cor_sig, icx], xerr=err_sig, color=color_list)
conditions_new = [i.split('-')[0] for i in conditions]
axs.set_yticks(np.arange(len(cor_sig)))
axs.set_yticklabels(np.array(conditions_new)[cor_sig])
axs.legend(axx,['ctrl','tab'])